In [1]:
import findspark
findspark.init()
from pyspark import SparkContext
from pyspark.sql.session import SparkSession
spark_context = SparkContext("local", "first_app")
spark = SparkSession(spark_context)
import plotly.graph_objects as go
from pyspark.sql import SQLContext
sql_context = SQLContext(spark_context)

In [2]:
text_file = sql_context.read.format("com.databricks.spark.csv").options(header='true', inferschema='true', quote='"', delimiter=',').load("/home/jovyan/work/Covid19.csv")
rdd_filter = text_file.rdd.map(tuple)
rdd_covid = rdd_filter.map(lambda word: (word[7], word[5]))
rdd_covid.take(5)

[('Afghanistan', 0),
 ('Afghanistan', 125),
 ('Afghanistan', 47),
 ('Afghanistan', 0),
 ('Afghanistan', 17)]

In [3]:
rdd_covid = rdd_covid.reduceByKey(lambda a,b: a+b)
print(rdd_covid.top(3))

[('Vietnam', 716), ('Venezuela', 14263), ('Thailand', 1248)]


In [4]:
rdd_covid = rdd_covid.filter(lambda word: word[0] in ["Cuba", "France", "Canada", "Singapore", "South_Korea"])
print(rdd_covid.collect())

[('Canada', 143649), ('Cuba', 5091), ('France', 453763), ('Singapore', 57576), ('South_Korea', 23045)]


In [5]:
covid_country = rdd_covid.map(lambda x: x[0])
covid_cases = rdd_covid.map(lambda x: x[1])
print(covid_country.collect())
print(covid_cases.collect())

['Canada', 'Cuba', 'France', 'Singapore', 'South_Korea']
[143649, 5091, 453763, 57576, 23045]


In [6]:
fig = go.Figure(data=go.Bar(x=covid_country.collect(),y=covid_cases.collect()))
fig.update_layout(title_text='Covid cases per country',title_font_size=30,
                  yaxis=dict(title='Total',title_font_size=25),
                  xaxis=dict(title='Country',title_font_size=25))
fig.update_traces(overwrite=True, marker={"opacity": 0.5})
fig.write_html('report5.html', auto_open=True)